In [0]:
import requests
import json
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql.functions import sum,avg,max,min,mean,count,col,when

In [0]:
spark.conf.set(
  "")
spark.conf.set(
  "")

In [0]:
def gradingSentiment(score):
    if(score == None):
      return 0
    if(float(score)>=0.5):
      return 1
    elif(float(score)<=-0.5):
      return -1
    else:
      return 0
udfsomefunc = F.udf(gradingSentiment, IntegerType())

In [0]:
tablename = ""
month = "mar"
startDate = 1
endDate = 31
for i in range(startDate, endDate+1):
  if i == 13:
    continue
  intermeiatefilename = "wasbs://"+month+"@sentimentdatacs4225.blob.core.windows.net/"+month+"-sentiment-NLTK-" + str(i) + ".csv"
  df = spark.read.csv(intermeiatefilename)
  df =df.withColumnRenamed("_c0","Date").withColumnRenamed("_c1","Language").withColumnRenamed("_c2","Country").withColumnRenamed("_c3","tweet").withColumnRenamed("_c4","sentimentScore")
  df2 = df.withColumn('sentimentDefine', udfsomefunc(df.sentimentScore))
  df2 = df2.groupby("Date","Country").agg(count(when(col("sentimentDefine") == 1, True)).alias("positive_Count"),count(when(col("sentimentDefine") == 0, True)).alias("netural_Count"),count(when(col("sentimentDefine") == -1, True)).alias("negative_Count"))

  df2.write.format("jdbc") \
    .mode("append") \
    .option("url", "") \
    .option("dbtable", tablename) \
    .option("user", "") \
    .option("password", "") \
    .save()